In [1]:
from pprint import pprint
import json
import os

import qcodes as qc

qc.set_mp_method('spawn')  # force Windows behavior on mac

qc.show_subprocess_widget()
from toymodel import AModel, MockGates, MockSource, MockMeter, AverageGetter, AverageAndRaw

model = AModel()
gates = MockGates('gates', model=model)
source = MockSource('source', model=model)
meter = MockMeter('meter', model=model)

station = qc.Station(gates, source, meter, meter.amplitude)
station.set_measurement(meter.amplitude)
c0, c1, c2, vsd = gates.chan0, gates.chan1, gates.chan2, source.amplitude


<IPython.core.display.Javascript object>

In [2]:
# Parameter snapshot
pprint(source.amplitude.snapshot())

{'__class__': 'qcodes.instrument.parameter.StandardParameter',
 'instrument': 'MockSource',
 'label': 'Source Amplitude (μV)',
 'name': 'amplitude',
 'ts': '2016-05-17 00:25:15',
 'units': '',
 'value': 0.1}


In [3]:
# Instrument snapshot
pprint(source.snapshot())

{'__class__': 'toymodel.MockSource',
 'functions': {},
 'name': 'source',
 'parameters': {'IDN': {'__class__': 'qcodes.instrument.parameter.StandardParameter',
                        'instrument': 'MockSource',
                        'label': 'IDN',
                        'name': 'IDN',
                        'ts': '2016-05-17 00:25:15',
                        'units': '',
                        'value': {'firmware': None,
                                  'model': None,
                                  'serial': None,
                                  'vendor': None}},
                'amplitude': {'__class__': 'qcodes.instrument.parameter.StandardParameter',
                              'instrument': 'MockSource',
                              'label': 'Source Amplitude (μV)',
                              'name': 'amplitude',
                              'ts': '2016-05-17 00:25:15',
                              'units': '',
                              'value': 0.1}}}


In [4]:
# Sweep_values snapshot
vals = c0[0:3:0.2]
pprint(vals.snapshot())

{'parameter': {'__class__': 'qcodes.instrument.parameter.StandardParameter',
               'instrument': 'MockGates',
               'label': 'Gate Channel 0 (mV)',
               'name': 'chan0',
               'ts': '2016-05-17 00:25:15',
               'units': '',
               'value': 0.0},
 'values': [{'first': 0.0, 'last': 2.8000000000000003, 'num': 15}]}


In [5]:
# Sweep_values snapshot
vals = c0.sweep(0,3,0.2)
pprint(vals.snapshot())

{'parameter': {'__class__': 'qcodes.instrument.parameter.StandardParameter',
               'instrument': 'MockGates',
               'label': 'Gate Channel 0 (mV)',
               'name': 'chan0',
               'ts': '2016-05-17 00:25:15',
               'units': '',
               'value': 0.0},
 'values': [{'first': 0.0, 'last': 3.0, 'num': 16}]}


In [6]:
# This is a qc.loop
lp = qc.Loop(c0.sweep(0,1,0.2),0)
lp.snapshot()

{'__class__': 'qcodes.loops.Loop',
 'delay': 0,
 'sweep_values': {'parameter': {'__class__': 'qcodes.instrument.parameter.StandardParameter',
   'instrument': 'MockGates',
   'label': 'Gate Channel 0 (mV)',
   'name': 'chan0',
   'ts': '2016-05-17 00:25:15',
   'units': '',
   'value': 0.0},
  'values': [{'first': 0.0, 'last': 1.0, 'num': 6}]}}

In [7]:
# After the loop has run, the dataset contains more info, and the loop that was run, became an active-loop
data = lp.run('data/dataset')
pprint(data.snapshot())

DataSet:
   mode     = DataMode.PULL_FROM_SERVER
   location = '2016-05-17/00-25-15'
   <Type>   | <array_id> | <array.name> | <array.shape>
   Setpoint | chan0      | chan0        | (6,)
   Measured | amplitude  | amplitude    | (6,)
started at 2016-05-17 00:25:16
{'__class__': 'qcodes.data.data_set.DataSet',
 'arrays': {'amplitude': {'__class__': 'qcodes.data.data_array.DataArray',
                          'action_indices': (0,),
                          'array_id': 'amplitude',
                          'instrument': 'MockMeter',
                          'is_setpoint': False,
                          'label': 'Current (nA)',
                          'name': 'amplitude',
                          'size': (6,),
                          'unit': None,
                          'units': ''},
            'chan0': {'__class__': 'qcodes.data.data_array.DataArray',
                      'action_indices': (),
                      'array_id': 'chan0',
                      'instrument':

In [8]:
# Here the loop is a qc.active_loop (due to the .each()) from the beginning
lp = qc.Loop(c0.sweep(0,1,0.2),0).each(qc.Wait(1))
pprint(lp.snapshot())

{'__class__': 'qcodes.loops.ActiveLoop',
 'actions': [{'delay': 1, 'type': 'Wait'}],
 'delay': 0,
 'sweep_values': {'parameter': {'__class__': 'qcodes.instrument.parameter.StandardParameter',
                                'instrument': 'MockGates',
                                'label': 'Gate Channel 0 (mV)',
                                'name': 'chan0',
                                'ts': '2016-05-17 00:25:15',
                                'units': '',
                                'value': 0.0},
                  'values': [{'first': 0.0, 'last': 1.0, 'num': 6}]}}


In [9]:
# When run, the dataset has a snapshot again
data = lp.run('data/dataset')
pprint(data.metadata)

Waiting for the previous background Loop to finish...
...done. Starting 2016-05-17/00-25-17
DataSet:
   mode     = DataMode.PULL_FROM_SERVER
   location = '2016-05-17/00-25-17'
   <Type>   | <array_id> | <array.name> | <array.shape>
   Setpoint | chan0      | chan0        | (6,)
started at 2016-05-17 00:25:17
{'data': {'__class__': 'qcodes.data.data_set.DataSet',
          'arrays': {'chan0': {'__class__': 'qcodes.data.data_array.DataArray',
                               'action_indices': (),
                               'array_id': 'chan0',
                               'instrument': 'MockGates',
                               'is_setpoint': True,
                               'label': 'Gate Channel 0 (mV)',
                               'name': 'chan0',
                               'size': (6,),
                               'unit': None,
                               'units': ''}},
          'base_location': '/Users/Merlin/Google '
                           'Drive/git/Qco

In [10]:
# When we add more actions:
lp = qc.Loop(c0.sweep(0,1,0.2),0).each(c0, c1, c2)
pprint(lp.snapshot())

{'__class__': 'qcodes.loops.ActiveLoop',
 'actions': [{'__class__': 'qcodes.instrument.parameter.StandardParameter',
              'instrument': 'MockGates',
              'label': 'Gate Channel 0 (mV)',
              'name': 'chan0',
              'ts': '2016-05-17 00:25:17',
              'units': '',
              'value': 1.0},
             {'__class__': 'qcodes.instrument.parameter.StandardParameter',
              'instrument': 'MockGates',
              'label': 'Gate Channel 1 (mV)',
              'name': 'chan1',
              'ts': '2016-05-17 00:25:15',
              'units': '',
              'value': 0.0},
             {'__class__': 'qcodes.instrument.parameter.StandardParameter',
              'instrument': 'MockGates',
              'label': 'Gate Channel 2 (mV)',
              'name': 'chan2',
              'ts': '2016-05-17 00:25:15',
              'units': '',
              'value': 0.0}],
 'delay': 0,
 'sweep_values': {'parameter': {'__class__': 'qcodes.instrument.pa

In [11]:
# When we add a second loop it is also just an action:
lp = qc.Loop(c0.sweep(0,1,0.2),0).loop(c1.sweep(0,1,0.2),0).each(c0, c1, c2)
pprint(lp.snapshot())

{'__class__': 'qcodes.loops.ActiveLoop',
 'actions': [{'__class__': 'qcodes.loops.ActiveLoop',
              'actions': [{'__class__': 'qcodes.instrument.parameter.StandardParameter',
                           'instrument': 'MockGates',
                           'label': 'Gate Channel 0 (mV)',
                           'name': 'chan0',
                           'ts': '2016-05-17 00:25:17',
                           'units': '',
                           'value': 1.0},
                          {'__class__': 'qcodes.instrument.parameter.StandardParameter',
                           'instrument': 'MockGates',
                           'label': 'Gate Channel 1 (mV)',
                           'name': 'chan1',
                           'ts': '2016-05-17 00:25:15',
                           'units': '',
                           'value': 0.0},
                          {'__class__': 'qcodes.instrument.parameter.StandardParameter',
                           'instrument': 'MockGa

In [12]:
# Now the snapshots get large
data = lp.run('data/dataset')
pprint(data.metadata)

Waiting for the previous background Loop to finish...
...done. Starting 2016-05-17/00-25-24
DataSet:
   mode     = DataMode.PULL_FROM_SERVER
   location = '2016-05-17/00-25-24'
   <Type>   | <array_id> | <array.name> | <array.shape>
   Setpoint | chan0_set  | chan0        | (6,)
   Setpoint | chan1_set  | chan1        | (6, 6)
   Measured | chan1      | chan1        | (6, 6)
   Measured | chan2      | chan2        | (6, 6)
   Measured | chan0      | chan0        | (6, 6)
started at 2016-05-17 00:25:24
{'data': {'__class__': 'qcodes.data.data_set.DataSet',
          'arrays': {'chan0': {'__class__': 'qcodes.data.data_array.DataArray',
                               'action_indices': (0, 0),
                               'array_id': 'chan0',
                               'instrument': 'MockGates',
                               'is_setpoint': False,
                               'label': 'Gate Channel 0 (mV)',
                               'name': 'chan0',
                           